In [17]:
from google.colab import drive
import warnings

drive.mount('/content/drive')
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os
import glob
import pandas as pd

item_path = os.path.join('drive/MyDrive', 'tfg/data/*')
files = [file for file in glob.glob(item_path) if file.endswith('processed.csv')]
item_name = max(files, key=os.path.getctime)

data = pd.read_csv(item_name)
data.head()

,log_duration,log_minhr,log_maxhr,log_avghr,log_trainingeffect,log_totenergyconsumption,log_hrabovetime,log_maxventilation,log_maxoxygenconsumption,log_maxbreathingfrequency,...,next_cancer_mama,next_cancer_laringe,next_cancer_prostata,next_cancer_pancreas,next_cancer_utero,next_cancer_vejiga,next_cancer_gastrico,next_cancer_linfoma,next_cancer_colon,next_cancer_tiroides
0,13802,100,122,118,1.8,236,8504,15,6,17,...,1,0,0,0,0,0,0,0,0,0
1,11346,108,122,119,2.0,171,5656,22,7,26,...,1,0,0,0,0,0,0,0,0,0
2,11367,53,161,95,3.6,293,10616,56,20,28,...,1,0,0,0,0,0,0,0,0,0
3,15777,98,117,116,1.8,306,11592,14,6,15,...,1,0,0,0,0,0,0,0,0,0
4,7986,71,132,96,2.2,242,7986,29,13,30,...,1,0,0,0,0,0,0,0,0,0


In [19]:
!pip install GPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import sys
import json
import joblib
import sklearn
import numpy as np
from shutil import rmtree
from tempfile import mkdtemp
from importlib import import_module
from GPy.models import GPRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.base import BaseEstimator, TransformerMixin
from GPy.kern import Linear, RBF, Matern32, Matern52, White
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.metrics import median_absolute_error, mean_squared_error, r2_score

class Columns(BaseEstimator, TransformerMixin):
    def __init__(self, names=None):
        self.names = names

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X):
        return X[self.names]

def build_kernel(kernel, ard, columns):
    if kernel == 'linear':
        kern = Linear(columns, ARD=ard)
    elif kernel == 'rbf':
        kern = RBF(columns, ARD=ard)
    elif kernel == 'matern32':
        kern = Matern32(columns, ARD=ard)
    elif kernel == 'matern52':
        kern = Matern52(columns, ARD=ard)
    elif kernel == 'linear-rbf':
        kern = Linear(columns, ARD=ard) + RBF(columns, ARD=ard)
    elif kernel == 'linear-matern32':
        kern = Linear(columns, ARD=ard) + Matern32(columns, ARD=ard)
    elif kernel == 'linear-matern52':
        kern = Linear(columns, ARD=ard) + Matern52(columns, ARD=ard)
    if ard:
        kern += White(columns)
    return kern

def capitalize_or_pass(string):
    if string != string.upper():
        capitalized = string.capitalize()
    else:
        capitalized = string
    return capitalized

def name_to_class(name):
    split = name.split('_')
    capitalized = [capitalize_or_pass(string) for string in split]
    class_ = ''.join(capitalized)
    return class_

def import_sklearn_module(step, method):
    if method in {'kernel_ridge'}:
        module_name = 'sklearn.kernel_ridge'
    elif method in {'lasso', 'elastic_net', 'ARD_regression', 'logistic_regression'}:
        module_name = 'sklearn.linear_model'
    elif method in {'random_forest_regressor', 'random_forest_classifier'}:
        module_name = 'sklearn.ensemble'
    elif method in {'gaussian_process_regressor'}:
        module_name = 'sklearn.gaussian_process'
    elif method in {'RBF', 'constant_kernel', 'white_kernel'}:
        module_name = 'sklearn.gaussian_process.kernels'
    else:
        module_name = f'sklearn.{step}'
    if module_name not in sys.modules:
        import_module(module_name)
    class_ = f'{module_name}.{name_to_class(method)}'
    return class_

def build_search_grid(model_name, non_binary_fields, binary_fields):
    with open(model_name) as file:
        specification = json.load(file)
    specification_steps = specification['steps']
    pipeline_steps = list()
    param_grid = dict()
    for step_name, step_methods in specification_steps:
        param_grid[step_name] = list()
        for index, (method_name, method_parameters) in enumerate(step_methods.items()):
            if method_name == 'passthrough':
                method_object = method_name
            else:
                method_class = import_sklearn_module(step_name, method_name)
                method_object = eval(method_class)()
                if method_name == 'standard_scaler':
                  method_object = FeatureUnion(transformer_list=[('numeric', make_pipeline(Columns(names=non_binary_fields), method_object)), ('binary', Columns(names=binary_fields))])
            if index == 0: 
                pipeline_steps.append((step_name, method_object))  
            param_grid[step_name].append(method_object)

            if method_parameters is None: continue
            for parameter_name, parameter_range in method_parameters.items():
                if isinstance(parameter_range, int) or isinstance(parameter_range, str):
                    parameter_range = [parameter_range]
                elif isinstance(parameter_range, dict):
                    if parameter_name == 'kernel': 
                        parameter_range = build_kernel_parameters(parameter_range, X.shape[1])
                param_grid[f'{step_name}__{parameter_name}'] = parameter_range
    return pipeline_steps, param_grid

def choose_parameter(parameter_range, selected_parameters, parameter_combinations):
    if len(parameter_range) == len(selected_parameters):
        parameter_combinations.append(selected_parameters)
        return

    last_selected_parameter = max(selected_parameters) if len(selected_parameters) != 0 else '0'
    for parameter, value_range in parameter_range.items():
        if (parameter not in selected_parameters and parameter > last_selected_parameter):
            for value in value_range:
                newly_selected_parameters = {parameter: value}
                newly_selected_parameters.update(selected_parameters)
                choose_parameter(parameter_range, newly_selected_parameters, parameter_combinations)  

def find_parameter_combinations(parameter_range):
    parameter_combinations = list()
    choose_parameter(parameter_range, dict(), parameter_combinations)
    return parameter_combinations 

def build_kernel_parameters(parameter_range, size):
    expression_elements = parameter_range['expression'].split(' ')
    input_expressions = [list()]
    index = 0
    for expression_element in expression_elements:
        if expression_element in {'+', '*', '**'}:
            output_expressions = [input_expression + [expression_element] for input_expression in input_expressions]
        else:
            try:
                float(expression_element)
                output_expressions = [input_expression + [expression_element] for input_expression in input_expressions]
            except:
                subparameters_range = dict([(subparameter_name, subparameter_range) if isinstance(subparameter_range, list) else (subparameter_name, [subparameter_range]) for subparameter_name, subparameter_range in parameter_range['parameters'][index].items()])
                kernel_class = import_sklearn_module(None, expression_element)
                parameter_combinations = find_parameter_combinations(subparameters_range)
                parameter_combinations = [{parameter: [value]*size if 'RBF' in kernel_class and isinstance(value, (float, int)) else value for parameter, value in parameter_combination.items()} for parameter_combination in parameter_combinations]
                index += 1

                output_expressions = [input_expression + [str(eval(kernel_class)(**parameter_combination))] for input_expression in input_expressions for parameter_combination in parameter_combinations]
        input_expressions = output_expressions
    parameter_range = [eval(' '.join(output_expression)) for output_expression in output_expressions]
    return parameter_range

def write_results(best_estimator, X_test, y_test, model, target, kernel, ard, previous, index, X_train, binary_fields, non_binary_fields):       
    epoch_path = os.path.join('drive/MyDrive', 'tfg/epoch')
    suffix = list()
    if kernel:
        suffix.append(kernel)
    if ard:
        suffix.append('ard')
    if previous:
        suffix.append('previous')
    suffix = f'-{"-".join(suffix)}' if suffix else ''
    epoch_name = os.path.join(epoch_path, f'{index}-{model}-{target[5:]}{suffix}.joblib')
    with open(epoch_name, mode='wb') as file:
        joblib.dump(best_estimator, file)   
    if model.startswith('gaussian-process'):
        y_pred = best_estimator.predict(X_test, full_cov=True)[0]
    else:
        y_pred = best_estimator.predict(X_test)
    if target.endswith('vo2pico'):
        y_pred[y_pred < 3.5] = 3.5
        y_pred[y_pred > 75] = 75
    elif target.endswith('wattpico'):
        y_pred[y_pred < 5] = 5
        y_pred[y_pred > 400] = 400  
    elif target.endswith('fcuvp') or target.endswith('fcucrp'):
        y_pred[y_pred < 30] = 30
        y_pred[y_pred > 220] = 220 

    with open(epoch_name, mode='rb') as file:
        best_estimator_joblib = joblib.load(file)
    if model.startswith('gaussian-process'):          
        y_pred_joblib = best_estimator_joblib.predict(X_test, full_cov=True)[0]
    else:
        y_pred_joblib = best_estimator_joblib.predict(X_test)
    if target.endswith('vo2pico'):
        y_pred_joblib[y_pred_joblib < 3.5] = 3.5
        y_pred_joblib[y_pred_joblib > 75] = 75
    elif target.endswith('wattpico'):
        y_pred_joblib[y_pred_joblib < 5] = 5
        y_pred_joblib[y_pred_joblib > 400] = 400
    elif target.endswith('fcuvp') or target.endswith('fcucrp'):
        y_pred_joblib[y_pred_joblib < 30] = 30
        y_pred_joblib[y_pred_joblib > 220] = 220       
    assert (y_pred == y_pred_joblib).all()

    mae = round(median_absolute_error(y_test, y_pred), 4)
    mse = round(mean_squared_error(y_test, y_pred), 4)
    r2 = round(r2_score(y_test, y_pred), 4)
    return mae, mse, r2
    
def evaluate_model_parameters(data, model, target, kernel=None, ard=False, previous=False, cv=10):    
    excluded = [field for field in data.columns if field.startswith('next')]
    if not previous:
        excluded += [field for field in data.columns if field.startswith('previous') and field not in {'previous_edad', 'previous_peso', 'previous_talla', 'previous_imc', 'previous_fco'} and not field.startswith('previous_cancer')]
    target = f'next_{target}'
    y = data[[target]]
    X = data.drop(excluded, axis=1)
    non_binary_fields = [field for field in X.columns if field not in {'log_personal_smoking', 'user_gender'} and not field.startswith('previous_cancer') and not field.startswith('next_cancer')]
    binary_fields = [field for field in X.columns if field not in non_binary_fields]

    model_path = os.path.join('drive/MyDrive', 'tfg/model')
    model_name = os.path.join(model_path, f'{model}.json')
    if model != 'gaussian-process':
        kernel = ard = None
        pipeline_steps, param_grid = build_search_grid(model_name, non_binary_fields, binary_fields)
        memory = mkdtemp()
        pipeline = Pipeline(pipeline_steps, memory=memory)
        
    rows, columns = X.shape
    folds = KFold(n_splits=cv, shuffle=True, random_state=42)
    suffix = list()
    if kernel:
        kernel_ = kernel.replace('rbf', 'RBF')
        kernel_ = kernel_.replace('-', ' + ')
        suffix.append(f'kernel = {kernel_}')
    if ard:
        suffix.append('ARD prior')
    if previous:
        suffix.append('previous ergospirometry data included')
    suffix = f', {", ".join(suffix)}' if suffix else ''
    target_ = target.replace('_', ' ')[5:]
    print(f'Training specifications: target = {target_}{suffix}')
    print('Fold'.ljust(10), 'MAE'.ljust(10), 'MSE'.ljust(10), 'R2')
    mae_total = mse_total = r2_total = 0
    for index, (train_index, test_index) in enumerate(folds.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        if model.startswith('gaussian-process'):
            kern = build_kernel(kernel, ard, columns)
            scaler = StandardScaler().fit(X_train[non_binary_fields])
            X_train = np.concatenate((scaler.transform(X_train[non_binary_fields]), X_train[binary_fields]), axis=1) 
            X_test = np.concatenate((scaler.transform(X_test[non_binary_fields]), X_test[binary_fields]), axis=1) 
            best_estimator = GPRegression(X_train, y_train, kern)
            best_estimator.optimize_restarts(num_restarts=cv, verbose=False, robust=True)
        else:
            grid_search = GridSearchCV(pipeline, param_grid=param_grid, n_jobs=-1, cv=cv)
            grid_search.fit(X_train, y_train)
            best_estimator = grid_search.best_estimator_
        mae, mse, r2 = write_results(best_estimator, X_test, y_test, model, target, kernel, ard, previous, index,  X_train, binary_fields, non_binary_fields)
        mae_total += mae
        mse_total += mse
        r2_total += r2
        print(f'{index+1}/{cv}'.ljust(10), str(mae).ljust(10), str(mse).ljust(10), str(r2).ljust(10))
    try:
        rmtree(memory)
    except:
        pass
    print('Average'.ljust(10), str(round(mae_total/cv, 4)).ljust(10), str(round(mse_total/cv, 4)).ljust(10), str(round(r2_total/cv, 4)).ljust(10))

def evaluate_model(data, model_parameters):
    print(f'Training {model_parameters["model"][0].replace("-", " ").replace("rbf", "RBF").replace("ard", "ARD")} model on {model_parameters["cv"][0]} cross-validation folds')
    for parameter_combination in find_parameter_combinations(model_parameters):
        evaluate_model_parameters(data, **parameter_combination)
        print()

model = 'kernel-ridge-linear' # random-forest, lasso, kernel-ridge-rbf, kernel-ridge-linear, gaussian-process, elastic-net, ard
target = 'fcucrp'
model_parameters = {'model': [model], 'target': [target], 'previous': [False, True], 'cv': [10]}
if model == 'gaussian-process':
    model_parameters['kernel'] = ['linear', 'rbf', 'matern32', 'matern52', 'linear-rbf', 'linear-matern32', 'linear-matern52']
    model_parameters['ard'] = [False, True]
evaluate_model(data, model_parameters)

Training kernel ridge linear model on 10 cross-validation folds
Training specifications: target = fcucrp
Fold       MAE        MSE        R2
1/10       3.2276     45.6595    -0.1331   
2/10       3.1336     58.0521    -0.3958   
3/10       3.0231     50.5851    -0.3051   
4/10       3.4454     45.0453    -0.1309   
5/10       3.2071     46.5363    -0.1145   
6/10       3.2437     55.2712    -0.2365   
7/10       3.3385     44.6605    -0.0273   
8/10       3.6997     82.2049    -0.9601   
9/10       3.4944     44.2281    -0.0522   
10/10      3.266      38.5781    0.1051    
Average    3.3079     51.0821    -0.225    

Training specifications: target = fcucrp, previous ergospirometry data included
Fold       MAE        MSE        R2
1/10       1.7929     15.0491    0.6265    
2/10       1.7984     23.5876    0.4329    
3/10       1.7399     19.2094    0.5044    
4/10       1.9399     17.2983    0.5657    
5/10       1.7349     14.3262    0.6569    
6/10       1.8216     22.0939    0.505